# Predicting Molecular Properties with Graph Neural Networks

<div style="background-color: #f0f8ff; border: 2px solid #4682b4; padding: 10px;">
<a href="https://colab.research.google.com/github/DeepTrackAI/DeepLearningCrashCourse/blob/main/Ch011_GNN/ec11_1_gnn/gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<strong>If using Colab/Kaggle:</strong> You need to uncomment the code in the cell below this one.
</div>

In [ ]:
# !pip install deeplay  # Uncomment if using Colab/Kaggle.

This notebook provides you with a complete code example to predict the properties of small molecules using graph convolutional layers with message passing.

<div style="background-color: #f0f8ff; border: 2px solid #4682b4; padding: 10px;">
<strong>Note:</strong> This notebook contains the Code Example 11-1 from the book  

**Deep Learning Crash Course**  
Benjamin Midtvedt, Jesús Pineda, Henrik Klein Moberg, Harshith Bachimanchi, Joana B. Pereira, Carlo Manzo, Giovanni Volpe  
No Starch Press, San Francisco (CA), 2025  
ISBN-13: 9781718503922  

[https://nostarch.com/deep-learning-crash-course](https://nostarch.com/deep-learning-crash-course)

You can find the other notebooks on the [Deep Learning Crash Course GitHub page](https://github.com/DeepTrackAI/DeepLearningCrashCourse).
</div>

## Understanding Graph Convolutions

Calculate the adjacency matrix of a simple cycle graph ...

In [ ]:
import numpy as np

node_num = 5

A = np.zeros((node_num, node_num))
for i in range(node_num):
    for j in range(node_num):
        if j % node_num == (i + 1) % node_num:
            A[i, j] = 1

... print the resulting adjacency matrix ...

In [ ]:
print(f"A: {A}")

... assign node attributes (or features) ...

In [ ]:
node_attributes = np.zeros(node_num)
node_attributes[0] = 1

... print the node attributes ...

In [ ]:
print(f"node_attributes: {node_attributes}")

... implement a function to perform a graph convolution ...

In [ ]:
def graph_convolution(A, node_attributes):
    """Calculate graph convolution."""
    conv = np.zeros(node_num)
    for i in range(node_num):
        for j in range(node_num):
            conv[j] = conv[j] + A[i, j] * node_attributes[i]
    return conv

... and apply the graph convolution.

In [ ]:
for c in range(5):
    node_attributes = graph_convolution(A, node_attributes)
    print(f"Convolution {c + 1}: {node_attributes}")

## Predicting Molecular Properties with Graph Convolutions

### Implementing a Graph Convolution Layer

Define a class to implement a graph convolution ...

In [ ]:
import torch

class GraphConvolution(torch.nn.Module):
    """Graph convolution"""

    def forward(self, A, node_attr):
        """Compute the graph convolution."""
        return torch.matmul(A, node_attr)

... and a class to implement the class convolution layer.

In [ ]:
import deeplay as dl

class GCL(dl.DeeplayModule):
    """Graph convolution layer."""

    def __init__(self, in_feats, out_feats):
        """Initialize graph convolution layer."""
        super().__init__()
        self.transform = dl.Layer(torch.nn.Linear, in_feats, out_feats)
        self.propagate = dl.Layer(GraphConvolution)
        self.update = dl.Layer(torch.nn.ReLU)

    def add_self_loops(self, A):
        """Add diagonal self-loop to adjacency matrix."""
        return A + torch.eye(A.size(0)).to(A.device)

    def normalize(self, A):
        """Normalize adjacency matrix."""
        node_degrees = torch.sum(A, dim=1)
        inv_sqrt_node_degrees = node_degrees.pow(-0.5)
        inv_sqrt_node_degrees[inv_sqrt_node_degrees == float("inf")] = 0
        diag_matrix = torch.diag(inv_sqrt_node_degrees)
        return torch.matmul(torch.matmul(diag_matrix, A), diag_matrix)

    def forward(self, A, node_attr):
        """Transform, propagate and update the node attributes."""
        A = self.normalize(self.add_self_loops(A))

        transformed_node_attr = self.transform(node_attr)
        propagated_node_attr = self.propagate(A, transformed_node_attr)
        updated_node_attr = self.update(propagated_node_attr)
        return updated_node_attr

### Using the ZINC Dataset

Download the ZINC dataset ...

In [ ]:
from torch_geometric.datasets import ZINC

train_set = ZINC(root="ZINC_dataset/", subset=True, split="train")
val_set = ZINC(root="ZINC_dataset/", subset=True, split="val")
test_set = ZINC(root="ZINC_dataset/", subset=True, split="test")

... implement a function to plot the adjacency matrix of a molecule ...

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
from torch_geometric.utils import to_dense_adj

def plot_molecule(molecule):
    """Plot adjacency matrix of a molecule."""
    node_attr = molecule["x"].numpy().squeeze()  # Atom type numbers.
    A = to_dense_adj(molecule["edge_index"]).numpy().squeeze(0)
    logP = molecule["y"].item()

    plt.imshow(A, cmap=colormaps["gray"].reversed())
    plt.title(f"LogP={np.round(logP, 2)}", fontsize=24)
    plt.xlabel("Atom type", fontsize=16)
    plt.xticks(np.arange(len(node_attr)), node_attr, fontsize=12)
    plt.ylabel("Atom type", fontsize=16)
    plt.yticks(np.arange(len(node_attr)), node_attr, fontsize=12)
    ax = plt.gca()
    ax.xaxis.set_ticks_position("bottom")
    plt.show()

... and use to visualize the structures of some molecules.

In [ ]:
for molecule_index in [2, 1235, 9887]:
    plot_molecule(molecule=train_set[molecule_index])

### Implementing a Graph Convolutional Network

Create a class to implement a graph convolutional network ...

In [ ]:
class GCN(dl.DeeplayModule):
    """Graph convolutional network."""

    def __init__(self, num_atoms, embed_dim, hidden_feats, out_feats):
        """Initialize graph convolutional network."""
        super().__init__()
        self.node_embed = dl.Layer(torch.nn.Embedding, num_atoms, embed_dim)
        self.blocks = dl.LayerList()
        for f_in, f_out in zip([embed_dim, *hidden_feats[:-1]],
                               hidden_feats):
            self.blocks.append(GCL(in_feats=f_in, out_feats=f_out))
        self.dense_top = dl.Sequential(
            dl.Layer(torch.nn.Linear, hidden_feats[-1], hidden_feats[-1] // 4),
            dl.Layer(torch.nn.ReLU),
            dl.Layer(torch.nn.Linear, hidden_feats[-1] // 4, out_feats),
        )

    def forward(self, G):
        """Predict graph properties."""
        G["node_attr"] = self.node_embed(G["node_attr"])
        for block in self.blocks:
            G["node_attr"] = block(G["A"], G["node_attr"])

        num_graphs = torch.max(G["graph_ids"]) + 1
        pooled_graph_attr = torch.zeros(
            num_graphs, G["node_attr"].shape[1], device=G["node_attr"].device,
        )
        pooled_graph_attr = pooled_graph_attr.scatter_add(
            0, G["graph_ids"][:, None].expand_as(G["node_attr"]),
            G["node_attr"],
        )
        node_counts = torch.bincount(G["graph_ids"])
        pooled_graph_attr = pooled_graph_attr / node_counts[:, None]

        return self.dense_top(pooled_graph_attr).squeeze()  # LogP.

... and instantiate it.

In [ ]:
gcn_model = GCN(
    num_atoms=28, embed_dim=64, hidden_feats=[64,] * 4, out_feats=1,
).create()

### Training the Graph Convolutional Network



Define the data loaders ...

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(dataset=train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(dataset=val_set, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=False)

... implement a class to compile, train, and evaluate the graph convolutional network ...

In [ ]:
class MolecularRegressor(dl.Regressor):
    """Regressor model for molecular property prediction."""

    def __init__(self, model, **kwargs):
        """Initialize molecular regressor."""
        super().__init__(model, **kwargs)

    def batch_preprocess(self, G):
        """Preprocess the graph batch for model input."""
        G["node_attr"] = G["x"].squeeze()
        G["A"] = to_dense_adj(G["edge_index"]).squeeze(0)
        G["graph_ids"] = G["batch"]
        return G.to(self.device)

    def forward(self, G):
        """Calculate model output for input graph batch."""
        return self.model(self.batch_preprocess(G))

... and train the graph convolutional network.

In [ ]:
import os
from lightning.pytorch.callbacks import ModelCheckpoint

gcn = MolecularRegressor(
    gcn_model, loss=torch.nn.L1Loss(), optimizer=dl.Adam(lr=1e-3),   
).create()
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", dirpath=os.path.join("models", "gcn"),
    filename="ZINC-{epoch:02d}-{val_loss:.2f}", auto_insert_metric_name=False,
)
trainer = dl.Trainer(max_epochs=400, callbacks=[checkpoint_callback])
trainer.fit(gcn, train_loader, val_loader)

### Evaluating the Trained Graph Convolutional Network

Evaluate the performance of the trained graph convolutional network ...

In [ ]:
import glob

model_paths = glob.glob(os.path.join("models", "gcn", "ZINC-*.ckpt"))
best_model_path = sorted(model_paths, key=os.path.getmtime)[-1]
gcn_best = MolecularRegressor.load_from_checkpoint(best_model_path,
                                                   model=gcn_model)
test_results = trainer.test(gcn_best, test_loader)

... obtain the predicted and actual logP ...

In [ ]:
logP_gts, logP_preds = [], []
for G in test_loader:
    logP_gts.append(G.pop("y"))
    logP_preds.append(gcn_best(G))
logP_gts = torch.cat(logP_gts).cpu().numpy()
logP_preds = torch.cat(logP_preds).detach().cpu().numpy()

... and plot the predicted logP versus their ground truth values.

In [ ]:
heatmap, xedges, yedges = np.histogram2d(logP_preds, logP_gts, bins=50)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.plot([min(logP_gts), max(logP_gts)], [min(logP_gts), max(logP_gts)], "r--")
plt.imshow(heatmap.T, extent=extent, origin="lower")
plt.xlabel("Predicted LogP"); plt.ylabel("True LogP")
plt.show()

## Predicting Molecular Properties with Message Passing

### Implementing a Message Passing Layer

Implement a class to implement a layer to perform the transform step ...

In [ ]:
class TransformLayer(torch.nn.Module):
    """Transform layer."""

    def __init__(self, hidden_feats):
        """Initialize the transform layer."""
        super().__init__()
        self.linear = torch.nn.LazyLinear(hidden_feats)
        self.activation = torch.nn.ReLU()

    def forward(self, G):
        """Compute messages by transforming node and edge attributes."""
        src_node_attr = G["node_attr"][G["edge_index"][0]]
        tgt_node_attr = G["node_attr"][G["edge_index"][1]]
        edge_attr = G["edge_attr"]

        msg = torch.cat([src_node_attr, tgt_node_attr, edge_attr], dim=-1)
        G["msg"] = self.activation(self.linear(msg))
        return G

... a class to implement a layer to perform the propagate step ...

In [ ]:
class PropagateLayer(torch.nn.Module):
    """Propagate layer."""

    def __init__(self, hidden_feats):
        """Initialize propagate layer."""
        super().__init__()
        self.hidden_feats = hidden_feats

    def forward(self, G):
        """Aggregate messages from neighboring nodes."""
        num_nodes = G["node_attr"].size(0)
        aggregated_msg = torch.zeros(num_nodes, self.hidden_feats,
                                     dtype=G["node_attr"].dtype,
                                     device=G["node_attr"].device)
        tgt_node_idxs = G["edge_index"][1].unsqueeze(1).expand_as(G["msg"])
        aggregated_msg = aggregated_msg.scatter_add(0, tgt_node_idxs, G["msg"])
        G["aggregated_msg"] = aggregated_msg
        return G

... a class to implement a layer to perform the update step ...

In [ ]:
class UpdateLayer(torch.nn.Module):
    """Update layer."""

    def __init__(self, hidden_feats):
        """Initialize update layer."""
        super().__init__()
        self.linear = torch.nn.LazyLinear(hidden_feats)
        self.activation = torch.nn.ReLU()

    def forward(self, G):
        """Update node attributes combining them with aggregated messages."""
        attr = torch.cat([G["node_attr"], G["aggregated_msg"]], dim=-1)
        G["node_attr"] = self.activation(self.linear(attr))
        G["edge_attr"] = G["msg"]
        return G

... and use them to define a message passing layer.

In [ ]:
mpl = dl.Sequential(
    dl.Layer(TransformLayer, hidden_feats=64),
    dl.Layer(PropagateLayer, hidden_feats=64),
    dl.Layer(UpdateLayer, hidden_feats=64),
).create()

print(mpl)

### Implementing a Message Passing Network

Create a class to implement a message passing network ...

In [ ]:
class MPN(dl.DeeplayModule):
    """Message passing network."""

    def __init__(self, num_atoms, num_edge_embed, embed_dim, hidden_feats,
                 out_feats):
        """Initialize message passing network."""
        super().__init__()
        self.node_embed = dl.Layer(torch.nn.Embedding, num_atoms, embed_dim)
        self.edge_embed = \
            dl.Layer(torch.nn.Embedding, num_edge_embed, embed_dim)
        self.blocks = dl.LayerList()
        for f_out in hidden_feats:
            mpl = dl.Sequential(
                dl.Layer(TransformLayer, hidden_feats=f_out),
                dl.Layer(PropagateLayer, hidden_feats=f_out),
                dl.Layer(UpdateLayer, hidden_feats=f_out),
            )            
            self.blocks.append(mpl)
        self.dense_top = dl.Sequential(
            dl.Layer(torch.nn.Linear, hidden_feats[-1], hidden_feats[-1] // 4),
            dl.Layer(torch.nn.ReLU),
            dl.Layer(torch.nn.Linear, hidden_feats[-1] // 4, out_feats),
        )

    def forward(self, G):
        """Predict graph properties."""
        G["node_attr"] = self.node_embed(G["node_attr"])
        G["edge_attr"] = self.edge_embed(G["edge_attr"])
        for block in self.blocks:
            G = block(G)

        num_graphs = torch.max(G["graph_ids"]) + 1
        pooled_graph_attr = torch.zeros(
            num_graphs, G["node_attr"].shape[1], device=G["node_attr"].device,
        )
        pooled_graph_attr = pooled_graph_attr.scatter_add(
            0, G["graph_ids"][:, None].expand_as(G["node_attr"]),
            G["node_attr"],
        )
        node_counts = torch.bincount(G["graph_ids"])
        pooled_graph_attr = pooled_graph_attr / node_counts[:, None]

        return self.dense_top(pooled_graph_attr).squeeze()  # LogP.

... instantiate the message passing network ...

In [ ]:
mpn_model = MPN(
    num_atoms=28, num_edge_embed=4, embed_dim=64, hidden_feats=[64,] * 4,
    out_feats=1,
).create()

... train the message passing network ...

In [ ]:
mpn = MolecularRegressor(
    mpn_model, loss=torch.nn.L1Loss(), optimizer=dl.Adam(lr=1e-3),   
).create()
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss", dirpath=os.path.join("models", "mpn"),
    filename="ZINC-{epoch:02d}-{val_loss:.2f}", auto_insert_metric_name=False,
)
trainer = dl.Trainer(max_epochs=400, callbacks=[checkpoint_callback])
trainer.fit(mpn, train_loader, val_loader)

... evaluate the performance of the trained message passing network ...

In [ ]:
model_paths = glob.glob(os.path.join("models", "mpn", "ZINC-*.ckpt"))
best_model_path = sorted(model_paths, key=os.path.getmtime)[-1]
mpn_best = MolecularRegressor.load_from_checkpoint(best_model_path,
                                                   model=mpn_model)
test_results = trainer.test(mpn_best, test_loader)

... obtaining the predicted and actual logP ...

In [ ]:
logP_gts, logP_preds = [], []
for G in test_loader:
    logP_gts.append(G.pop("y"))
    logP_preds.append(mpn_best(G))
logP_gts = torch.cat(logP_gts).cpu().numpy()
logP_preds = torch.cat(logP_preds).detach().cpu().numpy()

... and plot the predicted logP versus their ground truth values.

In [ ]:
heatmap, xedges, yedges = np.histogram2d(logP_preds, logP_gts, bins=50)
extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.plot([min(logP_gts), max(logP_gts)], [min(logP_gts), max(logP_gts)], "r--")
plt.imshow(heatmap.T, extent=extent, origin="lower")
plt.xlabel("Predicted LogP"); plt.ylabel("True LogP")
plt.show()